# Deep cortical reinforcement learning: Policy gradients + Q-learning + GAN


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [9]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [10]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [11]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [12]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [13]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [14]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [15]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.6213634468955913 -2.831909608650985
actions: 1 0
rewards: 1.0 1.0


In [16]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [17]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [19]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [20]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [21]:
# Data of the model
def model_input(state_size, lstm_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    reward = tf.placeholder(tf.float32, [], name='reward')
    # GRU: Gated Recurrent Units
    gru = tf.nn.rnn_cell.GRUCell(lstm_size) # hidden size
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    g_initial_state = cell.zero_state(batch_size, tf.float32) # feedback or lateral/recurrent connection from output
    d_initial_state = cell.zero_state(batch_size, tf.float32) # feedback or lateral/recurrent connection from output
    return states, actions, targetQs, reward, cell, g_initial_state, d_initial_state

In [22]:
# How to use batch-norm
#   x_norm = tf.layers.batch_normalization(x, training=training)

#   # ...

#   update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#   with tf.control_dependencies(update_ops):
#     train_op = optimizer.minimize(loss)

In [23]:
# training: Either a Python boolean, or a TensorFlow boolean scalar tensor (e.g. a placeholder). 
# Whether to return the output in: 
# training mode (normalized with statistics of the current batch) or 
# inference mode (normalized with moving statistics). 
# NOTE: make sure to set this parameter correctly, or else your training/inference will not work properly.

In [24]:
# MLP & Conv
# # Generator/Controller: Generating/prediting the actions
# def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
#     with tf.variable_scope('generator', reuse=reuse):
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=states, units=hidden_size)
#         bn1 = tf.layers.batch_normalization(h1, training=training)        
#         nl1 = tf.maximum(alpha * bn1, bn1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         bn2 = tf.layers.batch_normalization(h2, training=training)        
#         nl2 = tf.maximum(alpha * bn2, bn2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=action_size)        
#         #predictions = tf.nn.softmax(logits)

#         # return actions logits
#         return logits

In [25]:
# RNN generator or sequence generator
def generator(states, initial_state, cell, lstm_size, num_classes, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [26]:
# MLP & Conv
# # Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
# def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
#     with tf.variable_scope('discriminator', reuse=reuse):
#         # Fusion/merge states and actions/ SA/ SM
#         x_fused = tf.concat(axis=1, values=[states, actions])
        
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
#         bn1 = tf.layers.batch_normalization(h1, training=training)        
#         nl1 = tf.maximum(alpha * bn1, bn1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         bn2 = tf.layers.batch_normalization(h2, training=training)        
#         nl2 = tf.maximum(alpha * bn2, bn2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=1)        
#         #predictions = tf.nn.softmax(logits)

#         # return rewards logits
#         return logits

In [27]:
# RNN generator or sequence generator
def discriminator(states, actions, initial_state, cell, lstm_size, reuse=False): 
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        inputs = tf.layers.dense(inputs=x_fused, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=1)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [28]:
def model_loss(action_size, hidden_size, states, actions, targetQs, reward,
               cell, g_initial_state, d_initial_state):
    # G/Actor
    #actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_logits, g_final_state = generator(states=states, num_classes=action_size, 
                                              cell=cell, initial_state=g_initial_state, lstm_size=hidden_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    g_loss = tf.reduce_mean(neg_log_prob_actions * targetQs)
    
    # D/Critic
    #Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    Qs_logits, d_final_state = discriminator(states=states, actions=actions_logits, 
                                             cell=cell, initial_state=d_initial_state, lstm_size=hidden_size)
    rewards = reward * tf.ones_like(targetQs)
    d_lossR = tf.reduce_mean(tf.square(tf.nn.sigmoid(tf.reshape(Qs_logits, [-1])) - rewards))
    d_lossR_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.reshape(Qs_logits, [-1]),
                                                                          labels=rewards))
    d_lossQ = tf.reduce_mean(tf.square(tf.reshape(Qs_logits, [-1]) - targetQs))
    d_lossQ_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.reshape(Qs_logits, [-1]),
                                                                          labels=tf.nn.sigmoid(targetQs)))
    #d_loss = d_lossR_sigm + d_lossQ_sigm + d_lossR + d_lossQ
    d_loss = d_lossR_sigm + d_lossQ

    return actions_logits, Qs_logits, g_final_state, d_final_state, g_loss, d_loss, d_lossR, d_lossQ, d_lossR_sigm, d_lossQ_sigm

In [29]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize RNN
    # g_grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(g_loss, g_vars), clip_norm=5) # usually around 1-5
    # d_grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(d_loss, d_vars), clip_norm=5) # usually around 1-5
    g_grads=tf.gradients(g_loss, g_vars)
    d_grads=tf.gradients(d_loss, d_vars)
    g_opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(g_grads, g_vars))
    d_opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(d_grads, d_vars))
    
    # # Optimize MLP & CNN
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    #     g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
    #     d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [30]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.reward, cell, self.g_initial_state, self.d_initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_final_state, self.d_final_state, self.g_loss, self.d_loss, self.d_lossR, self.d_lossQ, self.d_lossR_sigm, self.d_lossQ_sigm = model_loss(
            action_size=action_size, hidden_size=hidden_size,
            states=self.states, actions=self.actions, cell=cell,
            targetQs=self.targetQs, reward=self.reward,  
            g_initial_state=self.g_initial_state, d_initial_state=self.d_initial_state)
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

In [31]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [32]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [33]:
# Training parameters
# Network parameters
state_size = 4                 # number of units for the input state/observation -- simulation
action_size = 2                # number of units for the output actions -- simulation
hidden_size = 64               # number of units in each Q-network hidden layer -- simulation
batch_size = 32                # number of samples in the memory/ experience as mini-batch size
learning_rate = 0.001          # learning rate for adam

In [34]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)
(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 1)


In [35]:
state = env.reset()
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

In [36]:
memory.buffer[0]

[array([-0.00183902, -0.00193909,  0.00719643, -0.01100595]),
 0,
 array([-0.0018778 , -0.1971635 ,  0.00697631,  0.28393882]),
 1.0,
 0.0]

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
from collections import deque
episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []
rates_list, d_lossR_list, d_lossQ_list = [], [], []
d_lossRsigm_list, d_lossQsigm_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(11111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state
        g_initial_state = sess.run(model.g_initial_state)
        d_initial_state = sess.run(model.d_initial_state)

        # Training steps/batches
        while True:
            action_logits, g_final_state, d_final_state = sess.run(
                fetches=[model.actions_logits, model.g_final_state, model.d_final_state], 
                feed_dict={model.states: np.reshape(state, [1, -1]),
                           model.g_initial_state: g_initial_state,
                           model.d_initial_state: d_initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append([g_initial_state, g_final_state,
                                  d_initial_state, d_final_state])
            total_reward += reward
            g_initial_state = g_final_state
            d_initial_state = d_final_state
            state = next_state
            if done is True:
                break

        # Episode total reward and success rate/prob
        episode_reward.append(total_reward) # stopping criteria
        rate = total_reward/ 500 # success is 500 points: 0-1
            
        # Training
        batch = memory.buffer
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        next_states = np.array([each[2] for each in batch])
        rewards = np.array([each[3] for each in batch])
        dones = np.array([each[4] for each in batch])
        rnn_states = memory.states
        g_initial_states = np.array([each[0] for each in rnn_states])
        g_final_states = np.array([each[1] for each in rnn_states])
        d_initial_states = np.array([each[2] for each in rnn_states])
        d_final_states = np.array([each[3] for each in rnn_states])
        nextQs_logits = sess.run(fetches = model.Qs_logits,
                                 feed_dict = {model.states: next_states, 
                                              model.g_initial_state: g_final_states[0].reshape([1, -1]),
                                              model.d_initial_state: d_final_states[0].reshape([1, -1])})
        nextQs = nextQs_logits.reshape([-1]) * (1-dones) # exploit
        #print(nextQs.shape, nextQs_logits.shape, dones.shape, rewards.shape)
        targetQs = rewards + (0.99 * nextQs)
        #print(targetQs.shape, rewards.shape, nextQs.shape)
        g_loss, d_loss, d_lossR, d_lossQ, d_lossRsigm, d_lossQsigm, _, _ = sess.run(
            fetches=[model.g_loss, model.d_loss, 
                     model.d_lossR, model.d_lossQ, 
                     model.d_lossR_sigm, model.d_lossQ_sigm,
                     model.g_opt, model.d_opt], 
            feed_dict = {model.states: states, model.actions: actions,
                         model.reward: rate, model.targetQs: targetQs,
                         model.g_initial_state: g_initial_states[0].reshape([1, -1]),
                         model.d_initial_state: d_initial_states[0].reshape([1, -1])})
 
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'rate:{:.4f}'.format(rate),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss),
              'dlossR:{:.4f}'.format(d_lossR),
              'dlossQ:{:.4f}'.format(d_lossQ),
              'dlossRsigm:{:.4f}'.format(d_lossRsigm),
              'dlossQsigm:{:.4f}'.format(d_lossQsigm))
        # Ploting out
        rewards_list.append([ep, np.mean(episode_reward)])
        rates_list.append([ep, rate])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        d_lossR_list.append([ep, d_lossR])
        d_lossQ_list.append([ep, d_lossQ])
        d_lossRsigm_list.append([ep, d_lossRsigm])
        d_lossQsigm_list.append([ep, d_lossQsigm])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:39.0000 rate:0.0780 gloss:0.6308 dloss:1.6666 dlossR:0.1630 dlossQ:1.0036 dlossRsigm:0.6630 dlossQsigm:0.7097
Episode:1 meanR:78.5000 rate:0.2360 gloss:0.6941 dloss:1.7042 dlossR:0.0749 dlossQ:1.0005 dlossRsigm:0.7037 dlossQsigm:0.6841
Episode:2 meanR:68.6667 rate:0.0980 gloss:0.6337 dloss:1.6769 dlossR:0.1524 dlossQ:1.0022 dlossRsigm:0.6747 dlossQsigm:0.7039
Episode:3 meanR:69.2500 rate:0.1420 gloss:0.5623 dloss:1.6525 dlossR:0.1024 dlossQ:1.0111 dlossRsigm:0.6414 dlossQsigm:0.7269
Episode:4 meanR:71.4000 rate:0.1600 gloss:0.7452 dloss:1.7364 dlossR:0.1390 dlossQ:0.9963 dlossRsigm:0.7402 dlossQsigm:0.6619
Episode:5 meanR:84.0000 rate:0.2940 gloss:0.4946 dloss:1.6689 dlossR:0.0225 dlossQ:1.0159 dlossRsigm:0.6530 dlossQsigm:0.7423
Episode:6 meanR:85.7143 rate:0.1920 gloss:0.4015 dloss:1.6258 dlossR:0.0454 dlossQ:1.0329 dlossRsigm:0.5929 dlossQsigm:0.7703
Episode:7 meanR:93.0000 rate:0.2880 gloss:0.3234 dloss:1.6663 dlossR:0.0073 dlossQ:1.0498 dlossRsigm:0.6165 dlossQsigm

Episode:65 meanR:220.6212 rate:0.4420 gloss:0.3450 dloss:1.7137 dlossR:0.0014 dlossQ:1.0245 dlossRsigm:0.6892 dlossQsigm:0.7629
Episode:66 meanR:221.4179 rate:0.5480 gloss:1.0894 dloss:1.9189 dlossR:0.0804 dlossQ:1.0090 dlossRsigm:0.9099 dlossQsigm:0.3093
Episode:67 meanR:221.6765 rate:0.4780 gloss:0.5802 dloss:1.6945 dlossR:0.0040 dlossQ:0.9942 dlossRsigm:0.7003 dlossQsigm:0.6567
Episode:68 meanR:222.6522 rate:0.5780 gloss:0.6951 dloss:1.6703 dlossR:0.0004 dlossQ:0.9886 dlossRsigm:0.6817 dlossQsigm:0.6258
Episode:69 meanR:222.5571 rate:0.4320 gloss:0.8429 dloss:1.7459 dlossR:0.0389 dlossQ:0.9820 dlossRsigm:0.7639 dlossQsigm:0.5609
Episode:70 meanR:222.5211 rate:0.4400 gloss:0.6884 dloss:1.7019 dlossR:0.0144 dlossQ:0.9870 dlossRsigm:0.7150 dlossQsigm:0.6351
Episode:71 meanR:222.5000 rate:0.4420 gloss:0.8202 dloss:1.7649 dlossR:0.0472 dlossQ:0.9795 dlossRsigm:0.7854 dlossQsigm:0.5270
Episode:72 meanR:223.3425 rate:0.5680 gloss:1.0805 dloss:1.9058 dlossR:0.0795 dlossQ:0.9921 dlossRsigm:0

Episode:129 meanR:277.4800 rate:0.5900 gloss:0.6331 dloss:1.8951 dlossR:0.0580 dlossQ:1.0501 dlossRsigm:0.8450 dlossQsigm:0.3144
Episode:130 meanR:278.5600 rate:0.5420 gloss:0.5960 dloss:2.1946 dlossR:0.1246 dlossQ:1.0896 dlossRsigm:1.1050 dlossQsigm:0.2205
Episode:131 meanR:279.6200 rate:0.5520 gloss:0.5737 dloss:2.2136 dlossR:0.1218 dlossQ:1.1055 dlossRsigm:1.1081 dlossQsigm:0.2120
Episode:132 meanR:280.3800 rate:0.5280 gloss:0.6075 dloss:2.2911 dlossR:0.1487 dlossQ:1.0742 dlossRsigm:1.2169 dlossQsigm:0.1909
Episode:133 meanR:281.4000 rate:0.5620 gloss:0.4443 dloss:2.2932 dlossR:0.1350 dlossQ:1.0808 dlossRsigm:1.2124 dlossQsigm:0.1669
Episode:134 meanR:281.6000 rate:0.5780 gloss:0.4432 dloss:2.2650 dlossR:0.1231 dlossQ:1.0963 dlossRsigm:1.1687 dlossQsigm:0.1675
Episode:135 meanR:281.9600 rate:0.5660 gloss:0.4688 dloss:2.3300 dlossR:0.1411 dlossQ:1.0594 dlossRsigm:1.2706 dlossQsigm:0.1459
Episode:136 meanR:281.7400 rate:0.4500 gloss:0.5910 dloss:1.8463 dlossR:0.1078 dlossQ:0.9010 dlos

Episode:193 meanR:195.7000 rate:0.3560 gloss:0.2273 dloss:5.3975 dlossR:0.4113 dlossQ:1.5489 dlossRsigm:3.8485 dlossQsigm:0.0544
Episode:194 meanR:194.5700 rate:0.2940 gloss:0.1266 dloss:6.3587 dlossR:0.4953 dlossQ:2.0018 dlossRsigm:4.3569 dlossQsigm:0.0598
Episode:195 meanR:193.9600 rate:0.2960 gloss:0.2070 dloss:6.1315 dlossR:0.4926 dlossQ:1.7563 dlossRsigm:4.3752 dlossQsigm:0.0563
Episode:196 meanR:193.3500 rate:0.3040 gloss:0.3308 dloss:6.6258 dlossR:0.4825 dlossQ:1.9790 dlossRsigm:4.6468 dlossQsigm:0.0583
Episode:197 meanR:191.8700 rate:0.2480 gloss:0.3239 dloss:7.2185 dlossR:0.5637 dlossQ:2.1000 dlossRsigm:5.1185 dlossQsigm:0.0601
Episode:198 meanR:191.2900 rate:0.3440 gloss:0.1450 dloss:6.9490 dlossR:0.4288 dlossQ:2.4476 dlossRsigm:4.5014 dlossQsigm:0.0653
Episode:199 meanR:190.9800 rate:0.3220 gloss:0.0502 dloss:6.9647 dlossR:0.4583 dlossQ:2.2309 dlossRsigm:4.7338 dlossQsigm:0.0628
Episode:200 meanR:190.3400 rate:0.3140 gloss:0.3576 dloss:6.8693 dlossR:0.4691 dlossQ:2.1345 dlos

Episode:258 meanR:152.7300 rate:0.2500 gloss:0.6292 dloss:9.8827 dlossR:0.5621 dlossQ:3.5916 dlossRsigm:6.2910 dlossQsigm:0.0740
Episode:259 meanR:152.9500 rate:0.1660 gloss:0.0574 dloss:12.6212 dlossR:0.6955 dlossQ:3.7641 dlossRsigm:8.8572 dlossQsigm:0.0861
Episode:260 meanR:152.6500 rate:0.2080 gloss:0.1505 dloss:12.5052 dlossR:0.6272 dlossQ:4.0800 dlossRsigm:8.4252 dlossQsigm:0.0884
Episode:261 meanR:152.8500 rate:0.2520 gloss:0.4603 dloss:9.5011 dlossR:0.5592 dlossQ:2.8834 dlossRsigm:6.6177 dlossQsigm:0.0660
Episode:262 meanR:153.5900 rate:0.2940 gloss:0.8816 dloss:10.0293 dlossR:0.4982 dlossQ:3.6720 dlossRsigm:6.3573 dlossQsigm:0.0753
Episode:263 meanR:153.9000 rate:0.2140 gloss:1.0398 dloss:10.9765 dlossR:0.6176 dlossQ:3.7302 dlossRsigm:7.2464 dlossQsigm:0.0794
Episode:264 meanR:153.8600 rate:0.1480 gloss:0.0043 dloss:13.2528 dlossR:0.7259 dlossQ:4.0882 dlossRsigm:9.1646 dlossQsigm:0.0916
Episode:265 meanR:153.2400 rate:0.1140 gloss:0.0140 dloss:12.6104 dlossR:0.7849 dlossQ:3.185

...

Episode:6376 meanR:33.1100 rate:0.0460 gloss:0.0205 dloss:5.6231 dlossR:0.8745 dlossQ:0.4799 dlossRsigm:5.1432 dlossQsigm:0.0871
Episode:6377 meanR:33.0900 rate:0.0440 gloss:0.2196 dloss:4.9223 dlossR:0.8685 dlossQ:0.3704 dlossRsigm:4.5519 dlossQsigm:0.0927
Episode:6378 meanR:33.0800 rate:0.0620 gloss:0.0744 dloss:7.8590 dlossR:0.8480 dlossQ:0.7267 dlossRsigm:7.1324 dlossQsigm:0.0607
Episode:6379 meanR:33.2400 rate:0.0880 gloss:0.0120 dloss:8.0138 dlossR:0.7972 dlossQ:0.4685 dlossRsigm:7.5453 dlossQsigm:0.0556
Episode:6380 meanR:33.1500 rate:0.0840 gloss:0.3866 dloss:7.8164 dlossR:0.7941 dlossQ:0.8184 dlossRsigm:6.9981 dlossQsigm:0.0640
Episode:6381 meanR:33.0400 rate:0.0560 gloss:0.0229 dloss:6.9106 dlossR:0.8198 dlossQ:0.7926 dlossRsigm:6.1180 dlossQsigm:0.0991
Episode:6382 meanR:33.1000 rate:0.0700 gloss:0.0664 dloss:6.7450 dlossR:0.7729 dlossQ:0.7140 dlossRsigm:6.0310 dlossQsigm:0.1146
Episode:6383 meanR:33.1600 rate:0.0820 gloss:0.0130 dloss:6.9113 dlossR:0.7778 dlossQ:0.5544 dlos

Episode:6440 meanR:33.4100 rate:0.0540 gloss:0.1523 dloss:7.1081 dlossR:0.8465 dlossQ:0.9595 dlossRsigm:6.1486 dlossQsigm:0.0841
Episode:6441 meanR:33.3100 rate:0.0540 gloss:0.0164 dloss:5.4002 dlossR:0.8302 dlossQ:0.3842 dlossRsigm:5.0160 dlossQsigm:0.1034
Episode:6442 meanR:33.3200 rate:0.0920 gloss:0.0022 dloss:6.8590 dlossR:0.7755 dlossQ:0.4416 dlossRsigm:6.4175 dlossQsigm:0.0691
Episode:6443 meanR:33.3900 rate:0.0680 gloss:0.0023 dloss:8.2079 dlossR:0.8422 dlossQ:0.5502 dlossRsigm:7.6577 dlossQsigm:0.0465
Episode:6444 meanR:33.3000 rate:0.0700 gloss:0.0035 dloss:7.7002 dlossR:0.8262 dlossQ:0.4008 dlossRsigm:7.2994 dlossQsigm:0.0573
Episode:6445 meanR:33.1600 rate:0.0560 gloss:0.1232 dloss:5.6915 dlossR:0.8049 dlossQ:0.7417 dlossRsigm:4.9499 dlossQsigm:0.1178
Episode:6446 meanR:33.3500 rate:0.0900 gloss:0.0047 dloss:8.6661 dlossR:0.8106 dlossQ:0.4094 dlossRsigm:8.2566 dlossQsigm:0.0423
Episode:6447 meanR:33.3800 rate:0.0640 gloss:0.3745 dloss:8.1293 dlossR:0.8362 dlossQ:1.5382 dlos

Episode:6505 meanR:38.5000 rate:0.0640 gloss:0.1931 dloss:11.5476 dlossR:0.8703 dlossQ:2.5378 dlossRsigm:9.0098 dlossQsigm:0.0378
Episode:6506 meanR:38.4600 rate:0.0560 gloss:0.0222 dloss:8.8542 dlossR:0.8810 dlossQ:1.5156 dlossRsigm:7.3386 dlossQsigm:0.0770
Episode:6507 meanR:38.5900 rate:0.1100 gloss:0.0902 dloss:8.9808 dlossR:0.7857 dlossQ:0.7826 dlossRsigm:8.1982 dlossQsigm:0.0411
Episode:6508 meanR:38.6600 rate:0.0660 gloss:0.0061 dloss:10.3955 dlossR:0.8644 dlossQ:1.1369 dlossRsigm:9.2587 dlossQsigm:0.0433
Episode:6509 meanR:38.8300 rate:0.0820 gloss:0.2772 dloss:8.2736 dlossR:0.8067 dlossQ:0.4812 dlossRsigm:7.7924 dlossQsigm:0.0560
Episode:6510 meanR:38.9100 rate:0.0740 gloss:0.0296 dloss:9.4278 dlossR:0.8508 dlossQ:0.6317 dlossRsigm:8.7961 dlossQsigm:0.0400
Episode:6511 meanR:39.0200 rate:0.0980 gloss:0.3422 dloss:8.0481 dlossR:0.7788 dlossQ:0.4521 dlossRsigm:7.5960 dlossQsigm:0.0554
Episode:6512 meanR:39.1400 rate:0.0740 gloss:0.1770 dloss:10.0981 dlossR:0.8498 dlossQ:0.6075 d

Episode:6571 meanR:46.0100 rate:0.0960 gloss:0.0079 dloss:9.2028 dlossR:0.8121 dlossQ:0.7993 dlossRsigm:8.4035 dlossQsigm:0.0399
Episode:6572 meanR:46.1200 rate:0.1080 gloss:0.0320 dloss:7.7851 dlossR:0.7777 dlossQ:0.4501 dlossRsigm:7.3350 dlossQsigm:0.0396
Episode:6573 meanR:46.4400 rate:0.1380 gloss:0.0322 dloss:8.2550 dlossR:0.7311 dlossQ:0.4329 dlossRsigm:7.8222 dlossQsigm:0.0347
Episode:6574 meanR:46.2000 rate:0.0640 gloss:0.2414 dloss:10.7303 dlossR:0.8689 dlossQ:1.9591 dlossRsigm:8.7713 dlossQsigm:0.0384
Episode:6575 meanR:46.2000 rate:0.0860 gloss:0.0063 dloss:7.7133 dlossR:0.8168 dlossQ:0.5193 dlossRsigm:7.1939 dlossQsigm:0.0412
Episode:6576 meanR:46.4300 rate:0.1480 gloss:0.1197 dloss:7.2988 dlossR:0.7065 dlossQ:0.4661 dlossRsigm:6.8327 dlossQsigm:0.0421
Episode:6577 meanR:46.4500 rate:0.0820 gloss:0.0131 dloss:9.4902 dlossR:0.8367 dlossQ:0.5638 dlossRsigm:8.9264 dlossQsigm:0.0375
Episode:6578 meanR:46.3300 rate:0.0720 gloss:0.1208 dloss:9.7243 dlossR:0.8561 dlossQ:0.5396 dlo

Episode:6636 meanR:53.5300 rate:0.1080 gloss:0.0044 dloss:7.1507 dlossR:0.7535 dlossQ:0.3351 dlossRsigm:6.8156 dlossQsigm:0.0628
Episode:6637 meanR:53.3200 rate:0.0880 gloss:0.0269 dloss:8.1639 dlossR:0.7923 dlossQ:0.2777 dlossRsigm:7.8862 dlossQsigm:0.0584
Episode:6638 meanR:53.5900 rate:0.1320 gloss:0.0545 dloss:9.2662 dlossR:0.7431 dlossQ:0.4955 dlossRsigm:8.7706 dlossQsigm:0.0370
Episode:6639 meanR:53.4000 rate:0.0860 gloss:0.1830 dloss:9.6181 dlossR:0.8270 dlossQ:0.6417 dlossRsigm:8.9764 dlossQsigm:0.0392
Episode:6640 meanR:53.6900 rate:0.1380 gloss:0.2701 dloss:7.1771 dlossR:0.7166 dlossQ:0.3412 dlossRsigm:6.8360 dlossQsigm:0.0501
Episode:6641 meanR:54.0000 rate:0.1520 gloss:0.1444 dloss:8.6300 dlossR:0.7033 dlossQ:0.5288 dlossRsigm:8.1011 dlossQsigm:0.0452
Episode:6642 meanR:54.1000 rate:0.0980 gloss:0.0003 dloss:7.4492 dlossR:0.7762 dlossQ:0.3302 dlossRsigm:7.1190 dlossQsigm:0.0576
Episode:6643 meanR:54.4300 rate:0.1340 gloss:0.2432 dloss:7.7212 dlossR:0.7302 dlossQ:0.3789 dlos

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

In [ ]:
eps, arr = np.array(d_lossR_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses R')

In [ ]:
eps, arr = np.array(d_lossQ_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses Q')

## Testing

Let's checkout how our trained agent plays the game.

In [34]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(100):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        # if total_reward == 500:
        #     break
                
# Closing the env
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.